In [21]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()

In [35]:
def process_text_data_v2( text_data, File_name_to_be_moved, request_id, current_timestamp, post_cur, table_stg):
    print(f"Processing text data inside process_text_data_v2")
    try:
        # Split the XML data into lines
        lines = text_data.splitlines()

        # Regular expression pattern to match the elements
        pattern = re.compile(r"<([^>]+)>([^<]*)</\1>")
        
        # Define a dictionary for mapping tags to column names
        tag_to_column = {
            'BICEPSVENDOR': 'VENDOR_NUMBER',
            'VENDOR_FACILITY': 'VENDOR_FACILITY',
            'CONNECTINGVENDORFAC': 'CONNECTING_VENDOR_FAC',
            'CONNECTINGVENDORNBR': 'CONNECTING_VENDOR_NBR',
            'TRANSACTIONID': 'TRANSACTIONID'
        }

        rows_count = 0

        # Loop through each line in the XML data
        for line in lines:
            # Skip empty lines or lines containing only whitespace or NUL character
            if not line.strip() or line.strip() == "NUL":
                continue

            # Define a dictionary to store values
            element_values = {}

            # Find all matches in the text data
            matches = re.findall(pattern, line)

            # Store the matches in the dictionary
            for tag, value in matches:
                element_values[tag] = value.strip()

            # Handle the special single tag 'CONVDRCHG' logic
            if '<CONVDRCHG>' in line:
                element_values['CONNECTING_VENDOR_TYPE'] = 'CONVDRCHG'
            
            # Handle 'BICEPSVENDOR' processing logic
            if 'BICEPSVENDOR' in element_values:
                bicepsvendor_value = element_values['BICEPSVENDOR']

                # Check if the value has at least 6 characters
                if len(bicepsvendor_value) >= 6:
                    # Extract the last 6 digits as the vendor number
                    vendor_number = bicepsvendor_value[-6:]

                    # Extract the remaining part to the left of the last 6 digits as the facility number
                    facility_value = bicepsvendor_value[:-6]
                    
                    # Left-pad the facility number to ensure it has 3 characters
                    facility_value = facility_value.zfill(3)

                    # Assign the extracted values to new keys
                    element_values['VENDOR_FACILITY'] = facility_value

                    # Update 'BICEPSVENDOR' to only contain the vendor number
                    element_values['BICEPSVENDOR'] = vendor_number
                else:
                    print('Error while extracting', element_values['BICEPSVENDOR'])

            # Replace column names with the ones from tag_to_column
            mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}

            # Include additional columns in the mapping
            mapped_values['CONNECTING_VENDOR_TYPE'] = element_values.get('CONNECTING_VENDOR_TYPE', '')
            mapped_values['REQUEST_ID'] = request_id
            mapped_values['CREATION_DATE'] = current_timestamp
            mapped_values['LAST_UPDATE_DATE'] = current_timestamp
            mapped_values['CREATED_BY'] = 'CSPOMS'
            mapped_values['LAST_UPDATED_BY'] = 'CSPOMS'
            mapped_values['FILE_NAME'] = File_name_to_be_moved
            mapped_values['PROCESS_STATUS'] = 'N'

            # Generate SQL INSERT statement
            columns_str = ",".join(mapped_values.keys())
            values_str = ",".join(f"'{v}'" for v in mapped_values.values())

            insert_query = f"INSERT INTO {table_stg} ({columns_str}) VALUES ({values_str});"
            post_cur.execute(insert_query)
            rows_count += post_cur.rowcount  # Increment the row count
            
        print(f"Processed file: {File_name_to_be_moved}, Number of rows inserted: {rows_count}")
    except Exception as e:
        print(f"Error processing text data:")
        print(f"Error: {e}")


In [36]:
# Example usage:
text_data = """
<BICEPSVENDOR>01006113</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000001</TRANSACTIONID>
                                                  
<BICEPSVENDOR>18430393</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000002</TRANSACTIONID>
                                                  
<BICEPSVENDOR>18430407</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000003</TRANSACTIONID>
                                                  
<BICEPSVENDOR>24046013</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000004</TRANSACTIONID>
                                                  
<BICEPSVENDOR>46040656</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000005</TRANSACTIONID>
                                                  
<BICEPSVENDOR>77010075</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000006</TRANSACTIONID>
                                                  
<BICEPSVENDOR>92246013</BICEPSVENDOR><CONNECTINGVENDORFAC>00</CONNECTINGVENDORFAC><CONNECTINGVENDORNBR>000000</CONNECTINGVENDORNBR><CONVDRCHG><TRANSACTIONID>V072209061053000000007</TRANSACTIONID>
                                                  
"""

# Call the function with sample data
process_text_data_v2( text_data, "FileTest.txt", "-999", "2024-09-04", post_cur, "xxpo_connecting_vendor_stg")
connection.commit()

Processing text data inside process_text_data_v2
Processed file: FileTest.txt, Number of rows inserted: 7


In [32]:
def check_vendor_master(post_cur):
    update_query = '''
                UPDATE xxpo_connecting_vendor_stg
            SET 
                PROCESS_STATUS = 'E',
                ERROR_MESSAGE = 'Record not found in XXPO_VENDOR_MASTER'
            WHERE  
            process_status = 'N'
            AND
            NOT EXISTS (
                SELECT 1 
                FROM xxpo_vendor_master vm
                WHERE 
                    vm.vendor_number = xxpo_connecting_vendor_stg.vendor_number
                    AND vm.vendor_facility = xxpo_connecting_vendor_stg.vendor_facility
            )
                   '''
    
    # Execute the update statement
    post_cur.execute(update_query)
    
    # Get the count of affected rows
    affected_rows = post_cur.rowcount
    
    return affected_rows

In [33]:
def check_active_vendors(post_cur):
    update_query = '''
            UPDATE xxpo_connecting_vendor_stg
            SET 
                PROCESS_STATUS = 'E',
                ERROR_MESSAGE = 'Record not Active in XXPO_VENDOR_MASTER'
            WHERE 
            process_status = 'N'
            AND
            NOT EXISTS (
                SELECT 1 
                FROM xxpo_vendor_master vm
                WHERE 
                    vm.vendor_number = xxpo_connecting_vendor_stg.vendor_number
                    AND vm.vendor_facility = xxpo_connecting_vendor_stg.vendor_facility
                    and vm.vendor_status = 'A'
            )
    '''
    
    # Execute the update statement
    post_cur.execute(update_query)
    
    # Get the count of affected rows
    affected_rows = post_cur.rowcount
    
    return affected_rows

In [34]:
error_vm_count = check_vendor_master(post_cur)
print(error_vm_count)
error_inactive = check_active_vendors(post_cur)
print(error_inactive)
proccesed_count = processed_p(post_cur)
print(proccesed_count)
connection.commit()

0
0
0


In [28]:
def processed_p(post_cur):
    update_query = '''
            UPDATE xxpo_connecting_vendor_stg
            SET 
                PROCESS_STATUS = 'P',
                Error_message = 'Processed'
            WHERE 
            process_status = 'N'
    '''
    
    # Execute the update statement
    post_cur.execute(update_query)
    
    # Get the count of affected rows
    affected_rows = post_cur.rowcount
    
    return affected_rows